In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Ejemplo pySparkSQL") \
    .config("spark.sql.warehouse.dir", "file:///D:/tmp/spark-warehouse") \
    .getOrCreate()

sc = spark.sparkContext

25/12/23 18:33:32 WARN Utils: Your hostname, MacBook-Pro-de-Sergio.local resolves to a loopback address: 127.0.0.1; using 172.19.109.126 instead (on interface en0)
25/12/23 18:33:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/23 18:33:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
%matplotlib inline 
from pyspark.sql import Row
import matplotlib.pyplot as plt
from pyspark.sql.functions import *
from pyspark.sql.functions import col
from functools import reduce

Guardamos el path del csv

In [3]:
csv_path = "/Users/vergara/Documents/Master/Computación Avanzada/Practicas_BigData/URL_phising_detector/phishing_features.csv"

Cargamos el csv en un dataframe e irprimos el schema del mismo

In [4]:
df = spark.read.csv(csv_path, inferSchema=True, header=True)
print("Elementos en DataFrame a partir de datos/personas.csv: " + str(df.count()) + "\nEsquema: ")
print (df.printSchema())
type(df)

Elementos en DataFrame a partir de datos/personas.csv: 160064
Esquema: 
root
 |-- url: string (nullable = true)
 |-- label: integer (nullable = true)
 |-- url_length: integer (nullable = true)
 |-- num_dots: integer (nullable = true)
 |-- has_https: integer (nullable = true)
 |-- has_ip: integer (nullable = true)
 |-- num_subdirs: integer (nullable = true)
 |-- num_params: integer (nullable = true)
 |-- suspicious_words: integer (nullable = true)
 |-- tld: string (nullable = true)
 |-- special_char_count: integer (nullable = true)
 |-- digits_count: integer (nullable = true)
 |-- entropy: double (nullable = true)

None


pyspark.sql.dataframe.DataFrame

In [ ]:
count_1 = df.filter(df['label'] == 1).count()
count_0 = df.filter(df['label'] == 0).count()

159244

Buscamos valores nulos en el dataset para tratarlo de forma adecuada

In [5]:
condition = reduce(
    lambda a, b: a | b,
    [col(c).isNull() for c in df.columns]
)

n = df.filter(condition).count()
df.filter(condition).show(n, truncate=False)

+------------------------+-----+----------+--------+---------+------+-----------+----------+----------------+----+------------------+------------+------------------+
|url                     |label|url_length|num_dots|has_https|has_ip|num_subdirs|num_params|suspicious_words|tld |special_char_count|digits_count|entropy           |
+------------------------+-----+----------+--------+---------+------+-----------+----------+----------------+----+------------------+------------+------------------+
|https://verify-att.app./|1    |24        |2       |1        |0     |3          |0         |1               |NULL|1                 |0           |3.688721875540867 |
|google.com              |0    |10        |1       |0        |0     |0          |0         |0               |NULL|0                 |0           |2.6464393446710157|
|youtube.com             |0    |11        |1       |0        |0     |0          |0         |0               |NULL|0                 |0           |3.095795255000934 |
|wik

In [6]:
df = df.withColumn(
    "tld",
    when(col("tld").isNull(),    
        element_at(split(lit("google.com"), r"\."), -1).alias("tld")
    ).otherwise(col("tld"))
)

df = df.cache()
df.count()

n = df.filter(condition).count()
df.filter(condition).show(n, truncate=False)



+---+-----+----------+--------+---------+------+-----------+----------+----------------+---+------------------+------------+-------+
|url|label|url_length|num_dots|has_https|has_ip|num_subdirs|num_params|suspicious_words|tld|special_char_count|digits_count|entropy|
+---+-----+----------+--------+---------+------+-----------+----------+----------------+---+------------------+------------+-------+
+---+-----+----------+--------+---------+------+-----------+----------+----------------+---+------------------+------------+-------+

